# Loop 1 LB Feedback Analysis

## Submission Results
- **CV Score**: 70.7343
- **LB Score**: 70.7343
- **Gap**: 0.0000 (perfect match!)

This is expected for an optimization problem - there's no train/test split, so CV = LB.

In [ ]:
# Current status
best_cv = 70.734327
best_lb = 70.734327
target = 68.931058
gap = best_lb - target

print(f"Best CV: {best_cv:.6f}")
print(f"Best LB: {best_lb:.6f}")
print(f"Target: {target:.6f}")
print(f"Gap to target: {gap:.6f} ({gap/target*100:.2f}%)")
print(f"\nImprovement needed: {gap:.6f} points")

In [ ]:
# Key insight: CV = LB for this optimization problem
# This means our local scoring is accurate
# The gap of 1.803 points (2.62%) must be closed through optimization

# What we know:
# 1. Pre-computed solutions are already well-optimized
# 2. fix_direction rotation optimization showed no improvement
# 3. bbox3 binary fails due to GLIBC version
# 4. C++ source code is available in Santa Claude kernel

print("Key findings:")
print("1. CV = LB (perfect match) - local scoring is accurate")
print("2. Pre-computed solutions are already well-optimized")
print("3. Need to run C++ optimizer to improve further")
print("4. C++ source code available: tree_packer_v21.cpp and bp.cpp")

In [ ]:
# Check if we can compile the C++ code
import subprocess

# Test g++ availability
result = subprocess.run(['g++', '--version'], capture_output=True, text=True)
print("G++ version:")
print(result.stdout.split('\n')[0])

# Check OpenMP support
result = subprocess.run(['g++', '-fopenmp', '--version'], capture_output=True, text=True)
if result.returncode == 0:
    print("\nOpenMP support: YES")
else:
    print("\nOpenMP support: NO")

In [ ]:
# Analyze score breakdown by n
import pandas as pd
import numpy as np

# Load the best solution
df = pd.read_csv('/home/code/santa_data/santa-2025.csv')

# Parse and calculate scores per n
from shapely.geometry import Polygon
from shapely import affinity
from decimal import Decimal, getcontext

getcontext().prec = 30

class ChristmasTree:
    def __init__(self, center_x="0", center_y="0", angle="0"):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        
        trunk_w, trunk_h = Decimal("0.15"), Decimal("0.2")
        base_w, mid_w, top_w = Decimal("0.7"), Decimal("0.4"), Decimal("0.25")
        tip_y, tier_1_y, tier_2_y = Decimal("0.8"), Decimal("0.5"), Decimal("0.25")
        base_y, trunk_bottom_y = Decimal("0.0"), -trunk_h
        
        initial_polygon = Polygon([
            (0, float(tip_y)),
            (float(top_w/2), float(tier_1_y)),
            (float(top_w/4), float(tier_1_y)),
            (float(mid_w/2), float(tier_2_y)),
            (float(mid_w/4), float(tier_2_y)),
            (float(base_w/2), float(base_y)),
            (float(trunk_w/2), float(base_y)),
            (float(trunk_w/2), float(trunk_bottom_y)),
            (float(-trunk_w/2), float(trunk_bottom_y)),
            (float(-trunk_w/2), float(base_y)),
            (float(-base_w/2), float(base_y)),
            (float(-mid_w/4), float(tier_2_y)),
            (float(-mid_w/2), float(tier_2_y)),
            (float(-top_w/4), float(tier_1_y)),
            (float(-top_w/2), float(tier_1_y)),
        ])
        
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x), yoff=float(self.center_y))

def get_score(trees, n):
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T for t in trees])
    side = max(xys.max(axis=0) - xys.min(axis=0))
    return side**2 / n

print("Calculating score breakdown...")

In [ ]:
# Calculate scores per n
scores_by_n = {}
for n in range(1, 201):
    group = df[df["id"].str.startswith(f"{n:03d}_")]
    if len(group) > 0:
        trees = []
        for _, row in group.iterrows():
            x = str(row["x"])[1:]
            y = str(row["y"])[1:]
            deg = str(row["deg"])[1:]
            trees.append(ChristmasTree(x, y, deg))
        scores_by_n[n] = get_score(trees, n)

# Find worst configurations (highest score contribution)
worst = sorted(scores_by_n.items(), key=lambda x: x[1], reverse=True)[:20]
print("Top 20 worst configurations (highest score contribution):")
for n, score in worst:
    print(f"  n={n:3d}: {score:.6f}")

print(f"\nTotal score: {sum(scores_by_n.values()):.6f}")

In [ ]:
# Calculate how much improvement is needed per configuration
target_score = 68.931058
current_score = sum(scores_by_n.values())
gap = current_score - target_score

print(f"Current score: {current_score:.6f}")
print(f"Target score: {target_score:.6f}")
print(f"Gap: {gap:.6f}")

# If we could improve each configuration by the same percentage
improvement_pct = gap / current_score * 100
print(f"\nUniform improvement needed: {improvement_pct:.2f}%")

# Or if we focus on worst configurations
print(f"\nIf we improve n=1 by 10%: {scores_by_n[1] * 0.1:.6f} saved")
print(f"If we improve n=1-10 by 5%: {sum(scores_by_n[n] for n in range(1,11)) * 0.05:.6f} saved")

In [ ]:
# Summary
print("="*60)
print("LOOP 1 SUMMARY")
print("="*60)
print(f"\nSubmission: 001_baseline_precomputed")
print(f"CV Score: {best_cv:.6f}")
print(f"LB Score: {best_lb:.6f}")
print(f"CV-LB Gap: 0.0000 (perfect match - expected for optimization problem)")
print(f"\nTarget: {target:.6f}")
print(f"Gap to target: {gap:.6f} ({gap/target*100:.2f}%)")
print(f"\nNext steps:")
print("1. Compile and run C++ optimizer (tree_packer_v21)")
print("2. Apply backward propagation (bp.cpp)")
print("3. If C++ fails, implement Python-based local search")